# Model Building with BoFire

This notebooks shows how to setup and analyze models trained with BoFire. It is stil WIP.

## Imports

In [9]:
from bofire.domain.features import (
    ContinuousInput,
    ContinuousOutput,
    InputFeatures,
    OutputFeatures,
)
from bofire.models.torch_models import SingleTaskGPModel
from bofire.utils.enum import RegressionMetricsEnum

## Problem Setup

For didactic purposes, we sample data from a Himmelblau benchmark function and use them to train a SingleTaskGP.

In [5]:
# Todo: replace this after JDs PR is ready.
input_features = InputFeatures(
        features=[
            ContinuousInput(key=f"x_{i+1}", lower_bound=-4, upper_bound=4)
            for i in range(2)
        ]
    )
output_features = OutputFeatures(features=[ContinuousOutput(key="y")])
experiments = input_features.sample(n=20)
experiments.eval("y=((x_1**2 + x_2 - 11)**2+(x_1 + x_2**2 -7)**2)", inplace=True)
experiments["valid_y"] = 1

## Cross Validation
### Run the cross validation

In [6]:
model = SingleTaskGPModel(
    input_features=input_features,
    output_features=output_features,
)
train_cv, test_cv = model.cross_validate(experiments, folds=5)

### Analyze the cross validation

Plots are added in a future PR.

In [7]:
# Performance on test sets
test_cv.get_metrics().describe()

,MAE,MSD,R2,MAPE
count,5.000000,5.000000,5.000000,5.000000
mean,29.124766,1674.292235,0.440958,2.397575
std,15.145197,1311.399571,0.766225,2.789861
min,11.724305,181.399644,-0.890757,0.101864
25%,17.023108,428.744328,0.479727,0.329180
50%,28.610077,1994.265951,0.780428,0.670958
75%,41.174075,2693.678985,0.875046,5.276980
max,47.092267,3073.372269,0.960345,5.608895


In [12]:
# performance on train sets
train_cv.get_metrics().describe()

,MAE,MSD,R2,MAPE
count,5.000000,5.000000,5.000000,5.000000
mean,9.652885,227.462662,0.957520,0.480095
std,6.789277,212.505371,0.037240,0.344671
min,0.045506,0.004077,0.915696,0.007622
25%,5.827419,53.026206,0.930102,0.247965
50%,10.898191,194.222770,0.949882,0.638519
75%,14.845984,408.338510,0.991920,0.644041
max,16.647322,481.721748,0.999999,0.862325
